In [1]:
import requests
import pandas as pd
from tqdm import tqdm 
from bs4 import BeautifulSoup
import os
import mimetypes
import numpy as np

# plot
import plotly.express as px
import plotly.graph_objects as go

# selenium method
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pyperclip
import time

# analysis
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder


# duckdb
import duckdb

/Users/stephensusanto/Desktop/Intro to Programming/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


##### Class Download File

In [2]:
class download_file:
    def status_bar_api(self, api_url:str, csv_filename:str = None, status:str = 'api'):
        # Stream the download
        if status == 'api':
            with requests.get(api_url, stream=True) as response:
                response.raise_for_status()
                total_size = int(response.headers.get('content-length', 0))
                chunk_size = 1024 * 1024  # 1 MB chunks
                
                chunks = []
                with tqdm(total=total_size, unit='B', unit_scale=True, desc='Downloading') as pbar:
                    for chunk in response.iter_content(chunk_size=chunk_size):
                        if chunk:
                            chunks.append(chunk)
                            pbar.update(len(chunk))
                
                # Combine chunks into a single bytes object
                content = b''.join(chunks)
            
            # Save CSV if filename is provided
            if csv_filename:
                with open(csv_filename, 'wb') as f:
                    f.write(content)
        else:
            response = requests.get(api_url, stream=True)
            response.raise_for_status()

            # Determine filename
            if "Content-Disposition" in response.headers:
                content_disposition = response.headers["Content-Disposition"]
                filename = content_disposition.split("filename=")[-1].strip('"')
            else:
                filename = os.path.basename(api_url)

            # If filename has no extension, try to guess from Content-Type
            if "." not in filename:
                content_type = response.headers.get("Content-Type", "")
                extension = mimetypes.guess_extension(content_type.split(";")[0].strip())
                if extension:
                    filename += extension

            # Get total file size for progress bar (in bytes)
            total_size = int(response.headers.get("content-length", 0))
            chunk_size = 8192  # 8 KB per chunk

            # Download with progress bar
            with open(filename, "wb") as f, tqdm(
                total=total_size, unit='B', unit_scale=True, desc=filename
            ) as progress_bar:
                for chunk in response.iter_content(chunk_size=chunk_size):
                    if chunk:
                        f.write(chunk)
                        progress_bar.update(len(chunk))
                

    # download from cer 
    def download_from_cer(self, url:str, csv_filename:str=None):
        """Extract API URL from CER datasets using Selenium and download CSV."""
        driver = webdriver.Chrome()
        driver.get(url)
        wait = WebDriverWait(driver, 15)

        # Find and click "Copy API URL" button
        button = wait.until(
            EC.element_to_be_clickable(
                (By.XPATH, "//button[.//span[contains(text(), 'Copy API URL')]]")
            )
        )
        button.click()
        time.sleep(1)  # wait for clipboard update

        api_url = pyperclip.paste()
        driver.quit()

        cer_code = url.split("/")[-1]
        api_url = f"https://api.cer.gov.au/datahub-public/v1/api/Dataset/NGER/dataset/{cer_code}.csv"
        print("Downloading from:", api_url)

        csv_filename = f"{cer_code}.csv"
        return self.status_bar_api(api_url=api_url, csv_filename=csv_filename)

    def download_cer_markets(self, url:str):
        url_header = url.split('/')[2]
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
            # Step 2: Find the button/link with the XLSX file
        div_tags = soup.find_all("div", class_="cer-accordion__body__item")
        for div in div_tags:
            a_tag = div.find("a", href=True)
            if a_tag:
                text = a_tag.get_text(strip=True).lower()  # normalize text
                if "csv" in text:
                    file_href = a_tag["href"]
                    full_url = f"https://www.{url_header}{file_href}"
                    #print(full_url)
                    self.status_bar_api(api_url=full_url,csv_filename=None, status='file')
    
    def download_abs(self, url:str):
        url_header = url.split('/')[2]
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        # Find the link for "Population and people"
        target_div = None
        for div in soup.find_all("div", class_="file-description-link-formatter"):
            h4 = div.find("h4")
            if h4 and "Economy and industry" in h4.text:
                target_div = div
                break
        if target_div:
            a_tag = target_div.find("a", href=True)
            relative_url = a_tag['href']
            download_url = f"https://{url_header}" + relative_url
            print("Found download URL:", download_url)
            self.status_bar_api(api_url=download_url,csv_filename=None, status='file')


<h1>Retrieve Data Set </h1>

In [3]:

download = download_file()

# first dataset
print("Processing Download First Dataset ....")
cer_url = "https://data.cer.gov.au/datasets/NGER/ID0243"
download.download_from_cer(url=cer_url)
# # second dataset
print("Processing Download Second Dataset ....")
cer_markets_url = "https://cer.gov.au/markets/reports-and-data/large-scale-renewable-energy-data"
download.download_cer_markets(url=cer_markets_url)
# third dataset
print("Processing Download Third Dataset ....")
abs_url = "https://www.abs.gov.au/methodologies/data-region-methodology/2011-24#data-downloads"
download.download_abs(url=abs_url)


Processing Download First Dataset ....


Downloading: 100%|██████████| 83.3k/83.3k [00:00<00:00, 69.8MB/s]


Processing Download Second Dataset ....


total-lgcs-and-capacity-accredited-power-stations-2025.csv: 100%|██████████| 421/421 [00:00<00:00, 2.32MB/s]
power-stations-and-projects-accredited.csv: 100%|██████████| 25.8k/25.8k [00:00<00:00, 33.5MB/s]
power-stations-and-projects-committed.csv: 100%|██████████| 1.89k/1.89k [00:00<00:00, 8.06MB/s]
power-stations-and-projects-probable.csv: 100%|██████████| 2.22k/2.22k [00:00<00:00, 4.88MB/s]
total-lgcs-rec-registry-0.csv: 100%|██████████| 45.4k/45.4k [00:00<00:00, 10.2MB/s]


Processing Download Third Dataset ....
Found download URL: https://www.abs.gov.au/methodologies/data-region-methodology/2011-24/14100DO0003_2011-24.xlsx


14100DO0003_2011-24.xlsx: 100%|██████████| 19.7M/19.7M [00:03<00:00, 6.20MB/s]


<h1>Data Integration and Cleaning Data </h1>

In [4]:
class cleaningData:
    def __init__(self):
        pass

    def show_null_data(self, data:pd.DataFrame):
        '''
            Showing Null Data for each column in a single dataframe

            Args:
                data: Dataframe to analyze

        '''
        null_count = data.isna().sum()
        null_percent = data.isna().sum() / len(data)
        null_percent = null_percent.apply(lambda x: f"{x:.1%}")
        print("Total Duplicated Count: ", data.duplicated().sum())
        results = pd.concat([null_count, null_percent], axis=1)
        results.columns = ['Null Total Count', 'Null Percentage']
        return results
    
    def split_categorical_numerical(self, data:pd.DataFrame):
        numerical_cols = data.select_dtypes(include=['int64', 'float64']).columns
        categorical_cols = data.select_dtypes(include=['object', 'category']).columns
        
        return numerical_cols, categorical_cols

class connectDB:
    def __init__(self):
        self.connection =  duckdb.connect('my_database.duckdb') 
    
    def create_new_table(self, df:pd.DataFrame, table_name:str):
        '''
            Creating New Table Based on Dataframe Columns


            Args:
                1. df: Dataframe or that we want to convert into database
                2. table_name: the name of the table that we want to create in duck db
        '''
        try:
            con = self.connection
            sql_types = {
                'object': 'VARCHAR',
                'int64': 'INTEGER',
                'float64': 'DOUBLE',
                'bool': 'BOOLEAN',
                'datetime64[ns]': 'TIMESTAMP'
            }

            columns_sql = []
            for col_name, dtype in df.dtypes.items():
                sql_type = sql_types.get(str(dtype), 'VARCHAR')  # Default to VARCHAR
                col_sql = f'"{col_name}" {sql_type}'
                columns_sql.append(col_sql)

            columns_str = ",\n    ".join(columns_sql)
            create_table_sql = f"CREATE TABLE {table_name} (\n    {columns_str}\n);"
            con.execute(create_table_sql)
            con.close()
        except Exception as err:
            con.close()
            print(err)
        
        return None
    
    def insert_data(self, df:pd.DataFrame, table_name: str, column_map: dict):
        '''
        Inserting database different column name with the csv

        Args:
            df: Dataframe to insert
            table_name: Name of the existing DuckDB table
            column_map: Mapping from dataframe columns to table columns
        
        column_map = {'df_col1': 'table_colA', 'df_col2': 'table_colB'}
        '''
        try:
            con = self.connection

            # Prepare SELECT clause using mapping
            select_clause = ", ".join([f'"{df_col}" AS "{table_col}"' for df_col, table_col in column_map.items()])
            
            # Register DataFrame as a relation
            con.register("df_temp", df)
            
            # Build INSERT SQL
            sql = f"""
            INSERT INTO {table_name} ({', '.join(column_map.values())})
            SELECT {select_clause} FROM df_temp
            """
            con.execute(sql)
            con.close()
        except Exception as err:
            con.close()
            print(err)
        


        


In [5]:

# first url file
df_nger = pd.read_csv("ID0243.csv")

# second url file
df_cer_1_approved = pd.read_csv("power-stations-and-projects-accredited.csv")
df_cer_1_commited = pd.read_csv("power-stations-and-projects-committed.csv")
df_cer_1_probable = pd.read_csv("power-stations-and-projects-probable.csv")


# third url file
df_abs = pd.ExcelFile("14100DO0003_2011-24.xlsx")
statistical_area = pd.read_excel(df_abs, 'Table 1', header=[5,6])
lga_area = pd.read_excel(df_abs, 'Table 2', header=[5,6])





#### Cleaning the NGER.CSV dataset and put it into DB

In [6]:
# module for Cleaning Data
cleaning_data = cleaningData()


df_nger_numerical, df_nger_categorical = cleaning_data.split_categorical_numerical(df_nger) 
df_nger.replace('-', np.nan, inplace=True)

df_nger_cleaning = df_nger.drop(columns=['Electricity production GJ', 'Total emissions t CO2 e'])
df_nger_cleaning.fillna("Unknown", inplace=True)

cleaning_data.show_null_data(df_nger_cleaning)


Total Duplicated Count:  0


/var/folders/2h/b49chrcn56z4tf_5y0zw27740000gn/T/ipykernel_86945/1614917072.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Unknown' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_nger_cleaning.fillna("Unknown", inplace=True)


,Null Total Count,Null Percentage
Reporting entity,0,0.0%
Facility name,0,0.0%
Type,0,0.0%
State,0,0.0%
Electricity production MWh,0,0.0%
Total scope 1 emissions t CO2 e,0,0.0%
Total scope 2 emissions t CO2 e,0,0.0%
Emission intensity t CO2 e MWh,0,0.0%
Grid connected,0,0.0%
Grid,0,0.0%


In [ ]:
df_cer_1_approved.replace('-', np.nan, inplace=True)

df_cer_1_approved
cleaning_data.show_null_data(df_cer_1_approved)

Total Duplicated Count:  0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280 entries, 0 to 279
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Accreditation code        280 non-null    object 
 1   Power station name        280 non-null    object 
 2   State                     280 non-null    object 
 3   Postcode                  280 non-null    int64  
 4   Installed capacity (MW)   280 non-null    float64
 5   Fuel Source (s)           280 non-null    object 
 6   Accreditation start date  280 non-null    object 
 7   Approval date             280 non-null    object 
dtypes: float64(1), int64(1), object(6)
memory usage: 17.6+ KB


In [21]:
df_cer_1_commited.replace('-', np.nan, inplace=True)

cleaning_data.show_null_data(df_cer_1_commited)

Total Duplicated Count:  0


,Null Total Count,Null Percentage
Project Name,0,0.0%
State,0,0.0%
MW Capacity,0,0.0%
Fuel Source,0,0.0%
Committed Date (Month/Year),0,0.0%


In [22]:
df_cer_1_probable.replace('-', np.nan, inplace=True)
cleaning_data.show_null_data(df_cer_1_probable)

Total Duplicated Count:  0


,Null Total Count,Null Percentage
Project Name,0,0.0%
State,0,0.0%
MW Capacity,0,0.0%
Fuel Source,0,0.0%


#### Process Creating Schema in Duck DB

In [8]:
# module related for database
database = connectDB()

column_name_facility = ['id','reporting_entity','facility_name','type','state',
                        'electricity_production_mwh','scope_1_emission', 
                        'scope_2_emission','emission_intesity_mwh','grid_connected',
                        'grid','primary_fuel', 'important_notes']

column_names_power_stations = ['id', 'accreditation_code', 'power_station_name','state','postcode',
                               'capacity_mw','source', 'accreditation_start_date','approval_date','commited_date','status']

# Create an empty DataFrame with the specified columns

In [9]:

print("----------")
df_cer_1_approved.replace('-', np.nan, inplace=True)

----------


In [10]:
print("----------")
#show_null_data(df_cer_1_commited)

----------


In [11]:
print("----------")
#show_null_data(df_cer_1_probable)



----------


In [12]:
print("----------")
#show_null_data(df_cer_2)

----------


In [13]:
#show_null_data(df_cer_3)

In [14]:

print("----------")
#show_null_data(statistical_area)


----------


In [15]:
print("----------")
#show_null_data(lga_area)

----------
